# Part 1

In [ ]:
import copy
import time
import math
import random
import numpy as np
import pandas as pd
from math import radians, cos, sin, asin, sqrt

# The dataset is loaded into a Pandas DataFrame.
dataset = pd.read_excel("JADS_BA_Assignment2_Resit_Data.xlsx")
# Several columns are irrelevant for this assignment and are dropped.
dataset = dataset.drop(columns=["Address", "Postal code", "City"])
# There is one NaN in the dataset. This is because the type is not provided for the headquarter.
# Therefore, an additional store type is created called "HQ", denoting the headquarter.
dataset.fillna("HQ", inplace=True)
# A list is created and all supermarket IDs are assigned to it. This list represents all unvisited
# supermarkets, which initially consists of all the supermarket IDs. All City Nrs are considered,
# except for the first element, since that one denotes the headquarter, which should be excluded
# from the unvisited_supermarkets list as it is the start/ending point for all routes.
unvisited_supermarkets = list(dataset["City Nr."][1:])

# The following formulas are used in the functions create_route and create_solution_q1 later on in order to
# construct the solution to this question. Each formula is documented by means of a docstring, as well as
# by means of comments inside the function.

def haversine(lon1, lat1, lon2, lat2):
    """Calculates and returns the haversine distance between two coordinates. Lon1 and lat1 denote the
    longitude and latitude for the first location respectively and lon2 and lat2 denote the longitude
    and latitude for the second location respectively."""
    # The radians function is mapped to the inputs to this function.
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # The differences in longitudes and latitudes between the two points are calculated.
    difference_lon = lon2 - lon1 
    difference_lat = lat2 - lat1
    a = sin(difference_lat/2)**2 + cos(lat1) * cos(lat2) * sin(difference_lon/2)**2
    c = 2 * asin(sqrt(a))
    # The radius of the earth is equal to 6371 kilometres.
    radius = 6371
    # The assignment text states that the result of the haversine function must be rounded to kilometers.
    return round(c * radius)

def distance_between_supermarkets(origin, destination):
    """Calculates the haversine distance between two locations. The inputs origin and destination
    are City Nrs."""
    # Retrieve the coordinates of the origin location from the dataset.
    origin_long, origin_lat = (dataset.loc[dataset["City Nr."] == origin]["Long"],
                               dataset.loc[dataset["City Nr."] == origin]["Lat"])
    # Retrieve the coordinates of the destination location from the dataset.
    destination_long, destination_lat = (dataset.loc[dataset["City Nr."] == destination]["Long"],
                                         dataset.loc[dataset["City Nr."] == destination]["Lat"])
    # Apply the haversine formula to these coordinates, which returns the distance between the two points.
    return haversine(origin_long, origin_lat, destination_long, destination_lat)

def closest_to_headquarters(unvisited_locations):
    """Returns a tuple, containing the City Nr. for the supermarket closest to the headquarter,
    along with the distance in kilometers between that supermarket and the headquarters."""
    # The following list comprehension calculates the distances for all supermarkets that are not visited
    # yet. Then, it selects the supermarket for which the distance to the headquarter is the least in the
    # list comprehension and returns it's City Nr., as well as its distance to the headquarters.
    return min([(supermarket, distance_between_supermarkets(0, supermarket)) for supermarket
                in unvisited_locations], key = lambda t: t[1])

def check_store_type(supermarket):
    """Returns a string, containing the type of supermarket to which the supermarket with the given City
    Nr. belongs."""
    # The type of the supermarket (i.e. whether it belongs to Jumbo, Coop or other) is retrieved from
    # the dataset by looking the supermarket up by its City Nr.
    return dataset.loc[dataset["City Nr."] == supermarket]["Type"].to_string(index=False).strip()

# Regarding the constraints, these can be split up into two separate checks that need to be performed:
# -- (Constraint 1) John works at most 11 hours. This means that the time spent between his departure
# from the HQ at the beginning of the day and his return at the HQ at the end of the day should be
# maximally 11 * 60 = 660 minutes.
maximum_working_hours = 11 * 60
# -- (Constraint 2) Every store can be visited between 09h00 and 17h00. Assuming that John arrives at
# the first location precisely at 09h00, this means that the sum of the driving time between the
# supermarkets (excluding driving times between the supermarkets and the headquarters) and the time
# spent in total at all the visits on that route should not exceed (17-9) * 60 = 480.
maximum_visit_time = (17-9) * 60

def check_constraint1(previous_location, route_distance, jumbo_supermarkets, coop_supermarkets, new_addition):
    """Checks whether a particular new insertion would be feasible according to constraint 1: John works
    at most 11 hours. Returns True if this constraint is satisfied or False in case it is not."""
    # route_distance contains the current distance of a route, which is maintained throughout the code
    # during the formation of a route. This prevents longer computational time, since the total distance
    # of the route does not have to be calculated again during each check of the constraints.
    driving_time = route_distance * (60/90)
    # visiting_time is calculated based on the current number of jumbo_supermarkets and coop_supermarkets
    # in the route. It contains the time spent during the visits so far.
    visiting_time = jumbo_supermarkets * 30 + coop_supermarkets * 20
    # Depending on the type of the store that is to be added, i.e. new_addition, an amount of either 20
    # or 30 minutes is spent during the visit. This results in different sums of the total time spent.
    # Since this sum is compared to the maximum limit of 11*60 minutes, either one of three checks is used:
    if check_store_type(new_addition) == "Jumbo":
        # The sum to check against the limit consists of the driving time and visiting time for the
        # supermarkets already in the route, the amount of minutes spent at the new_addition location,
        # the time it requires from the previously visited location to the new_addition, and the time it
        # requires to return to the headquarters from the new_addition.
        return driving_time + visiting_time + 30 + (distance_between_supermarkets(previous_location, new_addition)
                                                    * (60/90)) + (distance_between_supermarkets(new_addition, 0)
                                                                  * (60/90)) < maximum_working_hours
    if check_store_type(new_addition) == "Coop":
        # Sum is the same as above, with the difference being a different duration for the visit for new_addition.
        return driving_time + visiting_time + 20 + (distance_between_supermarkets(previous_location, new_addition)
                                                    * (60/90)) + (distance_between_supermarkets(new_addition, 0)
                                                                  * (60/90)) < maximum_working_hours
    if check_store_type(new_addition) == "Other":
        # Sum is the same as above, with the difference being a different duration for the visit for new_addition.
        return driving_time + visiting_time + 20 + (distance_between_supermarkets(previous_location, new_addition)
                                                    * (60/90)) + (distance_between_supermarkets(new_addition, 0)
                                                                  * (60/90)) < maximum_working_hours
    
def check_constraint2(route, route_distance, jumbo_supermarkets, coop_supermarkets, new_addition):
    """Checks whether a particular new insertion satisfies constraint 2: supermarkets can be visited between
    9h00 and 17h00. Since the sum of time spent at supermarkets and the driving times between the supermarkets
    cannot exceed (17-9) * 60 = 480 minutes."""
    # The visting_time denotes the time spent during the visits to the supermarkets. Since visits
    # to Jumbo and Coop/Other supermarkets require different visit durations, it can be calculated
    # as a function of the number of supermarkets of each type. Since jumbo_supermarkets and
    # coop_supermarkets are integers, the result will be rounded to minutes as required.
    visiting_time = (jumbo_supermarkets * 30) + (coop_supermarkets * 20)
    # Since the create_route function adds the distance between the first supermarket and the HQ
    # to the driving distance and this function neglects that distance (since it measures the time
    # spent beginning at the first visit), it needs to be subtracted from the distance so far. The
    # store to be added to the route (i.e. new_addition) also requires that John drives from the
    # last store in the current route to the to be inserted store, so this distance should be
    # included as well. The sum of distance is multiplied by (60/90) to calculate the driving time.
    # As stated in the assignment, the driving time must be rounded to minutes.
    driving_time = round((route_distance - distance_between_supermarkets(0, route[0]) +
                    distance_between_supermarkets(route[-1], new_addition)) * (60/90))
    # In addition to the time spent in the calculation above, John will spend some time at the
    # store to be added as well (i.e. the variable new_store_visit_time). This duration depends on
    # the type of the store, which is properly assigned to the variable using these if-statements:
    if check_store_type(new_addition) == "Jumbo":
        new_store_visit_time = 30
    if check_store_type(new_addition) == "Coop":
        new_store_visit_time = 20
    if check_store_type(new_addition) == "Other":
        new_store_visit_time = 20
    # Since the three components on the left-hand side of this inequality are all integers, the
    # resulting sum is rounded to minutes, as required by the assignment.
    return visiting_time + driving_time + new_store_visit_time < maximum_visit_time

def check_both_constraints(route, previous_location, route_distance, jumbo_supermarkets, coop_supermarkets, new_addition):
    """Checks whether both constraint 1 and constraint 2 have been satisfied."""
    # Both constraints have been specified in previous functions. This function aggregates these
    # two constraint checks into one function. First, the first constraint is checked.
    if check_constraint1(previous_location, route_distance, jumbo_supermarkets, coop_supermarkets, new_addition) == True:
        # If the first constraint is satisfied, the second constraint is checked.
        if check_constraint2(route, route_distance, jumbo_supermarkets, coop_supermarkets, new_addition) == True:
            # If both if-statements have been satisfied, both constraints have been satisfied and
            # the new_addition can be inserted. Hence, the function returns True.
            return True
        else:
            # In case constraint 2 is violated, the new_addition cannot be inserted into the route.
            return False
    else:
        # In case constraint 1 is violated, the new_addition cannot be inserted into the route.
        return False

def closest_to_supermarket(route, current_location, unvisited_locations, false_checks):
    """Returns the supermarket which is the closest to the current supermarket if such a location
    exists. If such a location exists, it returns a tuple with the supermarket ID and the distance
    between the current location and the closest supermarket. If such a location does not exist,
    the function returns False."""
    # If the length of false_checks (i.e. the previously selected nearest stores which did not
    # satisfy the constraints) is equal to the length of unvisited_locations, this means that
    # all unvisited_locations have been checked for being the nearest feasible store. In such
    # a case, there is no feasible nearest store for the current route and the route is complete.
    if len(false_checks) == len(unvisited_locations):
        return False
    else:
        # In case there are still unvisited stores to be checked for nearest location, the location ID
        # with the minimal distance from the current location is returned, along with the distance
        # from the current location to that selected minimal distance location.
        return min([(supermarket, distance_between_supermarkets(current_location, supermarket)) for
                    supermarket in unvisited_locations if supermarket != current_location if
                    supermarket not in false_checks], key = lambda t: t[1])

def create_route(unvisited_supermarkets):
    """Creates a new, single route (i.e. "day" in terminology of the assignment) which satisfies the
    constraints set by the assignment."""
    # The route distance is maintained by route_distance, which is initially zero for a new route.
    route_distance = 0
    # Two counters keep track of how many Jumbo and Coop/Other supermarkets are in the route. Note that
    # the variable coop_supermarkets keeps track of supermarkets of both Coop and Other type.
    # Initially, there are zero supermarkets in the route, so both are set to zero.
    jumbo_supermarkets = 0
    coop_supermarkets = 0
    # False checks maintains stores which have been selected for addition, but which did not satisfy
    # the constraints. Since the next store must be "the nearest store from x, say y, which can be added
    # after x in a feasible way" according to the assignment, it might be that the first selected y does
    # not satisfy the feasibility contraints, but the second, third, etc. closest to x does satsify
    # the constraints and should be inserted instead. False_checks maintains the selected stores
    # closest to the previous location which have previously failed the feasibility checks. Initially,
    # no stores have been checked yet and hence false_checks is empty.
    false_checks = []
    # The first location in the route is the closest supermarket to the headquarters, which is still
    # unvisited. The route is initiated, with the first visit being the closest unvisited store. Note that
    # closest_to_headquarters returns a tuple containing (city_nr, distance).
    route = [closest_to_headquarters(unvisited_supermarkets)[0]]
    # The first location in the route is removed from the list of unvisited supermarkets, since it has
    # now been visited.
    unvisited_supermarkets.remove(route[-1])
    # Depending on the type of the supermarket at the first visit, the appropriate counter is incremented.
    if check_store_type(route[-1]) == "Jumbo":
        jumbo_supermarkets += 1
    if check_store_type(route[-1]) in ["Coop", "Other"]:
        coop_supermarkets += 1
    # The route distance is updated to include the distance between HQ and the first location.
    route_distance += closest_to_headquarters(unvisited_supermarkets)[1]
    # As long as there are unvisited supermarkets, or the route has not been returned inside the body of
    # this while loop, the route construction for a particular day continues.
    while len(unvisited_supermarkets) != 0:
        # In terms of the assignment terminology, previous_location denotes the store x which is the
        # last store insterted into the route.
        previous_location = route[-1]
        # From this previous location, the nearest store (in the assigment terminology: y) is selected,
        # which can be added after x in a feasible way. Since previously selected nearest stores might have
        # failed the feasibility conditions, it should depend on false_checks. Also, it should not have
        # been visited before. Therefore, unvisited_locations and the current route should be considered.
        if closest_to_supermarket(route, previous_location, unvisited_supermarkets, false_checks) == False:
            # In case no nearest store exists, closest_to_supermarkets returns False, indicating that the
            # route is complete and can be returned.
            return route
        else:
            # If a nearest store exists, this nearest store's City Nr. is assigned to new_addition. Remember that
            # closest_to_supermarket returns a tuple containing the City Nr. at index 0 and the distance at index 1.
            new_addition = closest_to_supermarket(route, previous_location, unvisited_supermarkets, false_checks)[0]
            # The new_addition is checked against both constraints, to check whether it is feasible.
            if check_both_constraints(route, previous_location, route_distance, jumbo_supermarkets, coop_supermarkets, new_addition) == True:
                # If the addition of new_addition to the route is feasible, it is inserted.
                route.append(new_addition)
                # Depending on the type of the store, the appropriate counter is incremented.
                if check_store_type(new_addition) == "Jumbo":
                    jumbo_supermarkets += 1
                if check_store_type(new_addition) in ["Coop", "Other"]:
                    coop_supermarkets += 1
                # The route distance is updated to include the distance between the last and new location.
                route_distance += distance_between_supermarkets(previous_location, new_addition)
                # The newly added supermarket is removed from the list of unvisited supermarkets to ensure
                # that it cannot be considered in future routes.
                unvisited_supermarkets.remove(route[-1])
                # Since there is now a newly added store and false_checks depends on the previous location,
                # false_checks should be made empty again.
                false_checks = []
            else:
                # In case a new_addition does not satisfy both constraints, it is added to false_checks,
                # in order to ensure that it is not considered again for this route in the future.
                false_checks.append(new_addition)
                # The while loop then continues, causing a new store to be evaluated for insertion.
                continue
    # In case there are no unvisited locations left, the current route is returned.
    return route

def create_solution_q1():
    """Creates a solution to question 2.1 using the previously defined function."""
    # Overall_routes will store all individual routes, hence it will become a list of lists.
    # Initially, no individual routes have been created, so overall_routes is empty.
    overall_routes = []
    # As long as there are still unvisited locations, routes need to be created. For as long as
    # that is the case, the body of this while loop will continue to run.
    while len(unvisited_supermarkets) != 0:
        # An individual route (i.e. "day") is created by the function create_routes, which is
        # called while providing the list of stores which have not been visited as input.
        route = create_route(unvisited_supermarkets)
        # Once a new route has been found, it is added to the overall_routes.
        overall_routes.append(route)
        # Once a new route has been formed, some information about it is printed to the screen.
        print("Number of stores in route:", len(route), "-- Route:", route)
    # If there are no unvisited stores left, all stores have been placed in a route and hence
    # the list of routes can be returned.
    return overall_routes

# The solution to question 1 is calculated by calling the create_solution_q1 function.
solution_q1 = create_solution_q1()

Number of stores in route: 15 -- Route: [120, 115, 100, 91, 131, 74, 14, 8, 105, 122, 47, 46, 4, 103, 26]
Number of stores in route: 15 -- Route: [107, 25, 121, 118, 117, 123, 17, 36, 112, 113, 114, 64, 55, 104, 129]
Number of stores in route: 16 -- Route: [5, 106, 128, 12, 125, 99, 20, 132, 119, 9, 62, 1, 53, 39, 109, 88]
Number of stores in route: 15 -- Route: [84, 85, 110, 111, 6, 13, 72, 43, 68, 60, 19, 11, 10, 81, 97]
Number of stores in route: 14 -- Route: [42, 77, 76, 116, 18, 94, 35, 108, 87, 80, 83, 56, 98, 45]
Number of stores in route: 12 -- Route: [93, 70, 21, 69, 63, 16, 3, 95, 96, 34, 37, 71]
Number of stores in route: 14 -- Route: [23, 22, 24, 92, 58, 130, 90, 73, 78, 44, 75, 127, 49, 51]
Number of stores in route: 14 -- Route: [89, 86, 61, 57, 54, 52, 2, 66, 67, 59, 124, 133, 82, 101]
Number of stores in route: 15 -- Route: [7, 38, 40, 41, 32, 28, 31, 29, 30, 33, 50, 48, 79, 15, 65]
Number of stores in route: 3 -- Route: [102, 27, 126]


# Part 2

In [ ]:
def count_store_types(route):
    """Returns the number of Jumbo supermarkets and Coop/Other supermarkets in a given route."""
    # Initially, the counters for the number of supermarkets are set to zero.
    nr_jumbo_supermarkets = 0
    nr_coop_supermarkets = 0
    # Iterating through a route, the appropriate counters are incremented depending on the store type
    # of each individual destination within the route.
    for destination in route:
        if check_store_type(destination) == "Jumbo":
            nr_jumbo_supermarkets += 1
        if check_store_type(destination) in ["Coop", "Other"]:
            nr_coop_supermarkets += 1
    return nr_jumbo_supermarkets, nr_coop_supermarkets

def calculate_distance_full_route(route):
    """Calculates the distance for the full route, thereby taking into account the time required
    to travel from the HQ to the first supermarket and the time required to travel from the last
    supermarket back to the HQ."""
    # Initially, the distance is set to zero, since no distances have been calculated so far.
    driving_distance = 0
    # The distance between the headquarters and the first visit is added to the distance.
    driving_distance += distance_between_supermarkets(0, route[0])
    # The destinations in the route are considered pair-wise, so a list with the pairs is created.
    # Hence, if a route is [a, b, c, d], then destination_pairs is [(a,b), (b,c), (c,d)].
    destination_pairs = list(zip(route, route[1:]))
    for pair in destination_pairs:
        # For each pair of visits, the distance is calculated and added to driving_distance.
        driving_distance += distance_between_supermarkets(pair[0], pair[1])
    # At last, the distance between the last visit and the headquarters is added to the distance.
    driving_distance += distance_between_supermarkets(route[-1], 0)
    return driving_distance

def calculate_driving_time_full_route(route):
    """Calculates the driving time for the full route, thereby taking into account the
    time required to travel from the HQ to the first supermarket and the time required
    to travel from the last supermarket back to the HQ."""
    # The distance is multiplied by (60/90) in order to calculate the driving time from the distance.
    # As given by the assignment, the driving time must be rounded to minutes.
    return round(calculate_distance_full_route(route) * (60/90))

def calculate_driving_time_between_supermarkets(route):
    """Calculates the driving time for the route, thereby excluding the time required
    to travel from the HQ to the first supermarket and the time required to travel
    from the last supermarket back to the HQ."""
    driving_distance = 0
    # The destinations in the route are considered pair-wise, so a list with the pairs is created.
    # Hence, if a route is [a, b, c, d], then destination_pairs is [(a,b), (b,c), (c,d)].
    destination_pairs = list(zip(route, route[1:]))
    for pair in destination_pairs:
        # For each pair of visits, the distance is calculated and added to the driving distance.
        driving_distance += distance_between_supermarkets(pair[0], pair[1])
    # The distance is multiplied by (60/90) in order to calculate the driving time from the distance.
    # As given by the assignment, the driving time must be rounded to minutes.
    return round(driving_distance * (60/90))

def check_feasibility(set_of_routes):
    """Returns True if all routes in the provided set of routes comply with the requirements that the
    total duration of the day can last 11h at maximum and that the stores should be visited between 9-17h."""
    # Iterating through the set of routes, every route is checked for feasibility individually.
    for route in set_of_routes:
        # First, the number of stores for each type are determined by calling count_store_types.
        nr_jumbos, nr_coops = count_store_types(route)
        # Using the number of stores for each type, the total visiting time is calculated.
        visiting_time = (nr_jumbos * 30) + (nr_coops * 20)
        # The driving time excluding headquarters is calculated by calling the appropriate function.
        driving_time_excl_hq = calculate_driving_time_between_supermarkets(route)
        # The sum of visiting time, plus the driving time (excluding the time required to drive from the
        # HQ to the first location and the time required to drive from the last location back to the HQ)
        # should not exceed the maximum visiting time for the supermarkets (i.e. between 17h-9h).
        if visiting_time + driving_time_excl_hq <= maximum_visit_time:
            # The driving time between the supermarkets is calculated, thereby including the time required
            # to travel from the HQ to the first location and the last location to the HQ.
            driving_time_incl_hq = calculate_driving_time_full_route(route)
            # The visiting time, plus the driving time, including the time to drive to the HQ, should not
            # exceed the maximum hours of work allowed for John.
            if visiting_time + driving_time_incl_hq <= maximum_working_hours:
                # If both if-statements (i.e. constraints) have been met, the loop continues and the next
                # route is checked for feasibility.
                continue
            else:
                # In case the if-statement is not satisfied, a condition for feasibility has not been met
                # and hence the feasibility of the set of routes as a whole is not met.
                return False
        else:
            # In case the if-statement is not satisfied, a condition for feasibility has not been met
            # and hence the feasibility of the set of routes as a whole is not met.
            return False
    # If the loop has been completed and no False has been returned due to if-statements not being satisfied,
    # all routes are compliant with the constraints and hence are feasible. Then, True is returned.
    return True

def calculate_total_distance(set_of_routes):
    """Returns the total distance in kilometres for a given set of routes."""
    # Initially, no distances have been calculated so total_distance is set to zero.
    total_distance = 0
    for route in set_of_routes:
        # Iterating over the routes, the full distance for every route is calculated and added to the 
        # total distance so far.
        total_distance += calculate_distance_full_route(route)
    # Once the distances of all routes have been added, total_distance is returned.
    return total_distance

def find_index_in_sublist(set_of_routes, search_value):
    """Given a set of routes and a location to be searched for, this function returns
    the route nr. and the position in which the searched location is found."""
    for route in set_of_routes:
        # Iterating over the routes, the search_value is searched for in the route.
        if search_value in route:
            # Once a route contains the search value, the function returns the index of the particular
            # route in the set of routes (i.e. the route number), and the position in the route.
            return (set_of_routes.index(route), route.index(search_value))
        
def propose_swap(set_of_routes):
    """Given a set of routes (i.e. the current state), this function returns a new set of routes
    where two stores have been swapped, along with the store numbers of the swapped locations."""
    # In order to prevent unwanted permanent changes to the input, a deepcopy is made of the input.
    current_state = copy.deepcopy(set_of_routes)
    # Two stores should be randomly selected to be swapped with each other. This is done by drawing
    # two numbers randomly from the range 1 up to and including 133, representing City Nrs.
    swap_store1, swap_store2 = tuple(random.sample(range(1, 134), 2))
    # Given these two City Nrs for swapping, their locations in the current routes are searched. Their
    # location consists of the route number and position in that particular route.
    swap_pos1 = find_index_in_sublist(current_state, swap_store1)
    swap_pos2 = find_index_in_sublist(current_state, swap_store2)
    # A swap is made by interchanging the elements at the positions defined above.
    current_state[swap_pos1[0]][swap_pos1[1]], current_state[swap_pos2[0]][swap_pos2[1]] = current_state[swap_pos2[0]][swap_pos2[1]], current_state[swap_pos1[0]][swap_pos1[1]]
    # The updated (i.e. swapped) current state, and the City Nrs of the swapped stores are returned.
    return current_state, swap_store1, swap_store2

def update_tabu_list(tabu_list, location1, location2):
    """Updates the tabu list by checking whether the length of the list exceeds the specified
    maximum length of 50. It then inserts the most recent swaps to the tabu list."""
    if len(tabu_list) == 50:
        # The tabu list should not be longer than 50, therefore the last-most element in the tabu list is
        # popped (i.e. removed) from the tabu list once the length of the list is equal to 50 and after
        # insertion of the new element (i.e. tuple containing the latest swapped stores) into the tabu list.
        tabu_list.insert(0, (location1, location2))
        tabu_list.pop(-1)
    else:
        # In case the tabu list does not have a length of 50, the tuple containing the latest swapped stores
        # is simply inserted at the beginning of the tabu list at index 0.
        tabu_list.insert(0, (location1, location2))
    # After the insertion (and possibly deletion) have been performed, the updated tabu list is returned.
    return tabu_list

def check_tabu_list(tabu_list, location1, location2):
    """Returns True when the combination of location 1 and location 2 is not in the Tabu
    list and returns False otherwise (i.e. if they are in the tabu list)."""
    # Since swapping (a, b) is the same as swapping (b, a), both versions have to be checked for in the tabu
    # list. Therefore, two if-statements are required, each checking one of these two options.
    if (location1, location2) in tabu_list:
        return False
    if (location2, location1) in tabu_list:
        return False
    # In case False has not been returned in the previous if-statements, the swap of location 1 and location 2
    # is not in the tabu list and hence True is returned, indicating that the swap is not tabu.
    return True

def check_stopping_condition(nr_steps, start_time_q2):
    """Returns True when the stopping conditions have not been met yet and returns False
    if the stopping conditions have been met."""
    # There are two stopping conditions: (1) there are 100 steps (in the code considered as nr_steps) without
    # improvement and (2) exceeding the total limit time of 1,000 seconds. nr_steps refers to the number
    # of steps without improvements, which is maintained throughout the code.
    max_number_steps = 100
    if nr_steps > max_number_steps:
        # If nr_steps is larger than 100, condition 1 (listed above) has been violated, and False is returned.
        return False
    # The current time is given by the time method from the time module. start_time_q2 is the start time of the
    # function constructing the final answer to question 2 of the assignment. This variable is initiated by
    # time.time() as well at the beginning of the call to function create_solution_q2. Once the current time,
    # contained in variable called "now", exceeds the start time by more than 1,000 seconds, the condition is
    # violated and as a result returns False, to identify that the algorithm should be stopped.
    max_time_limit = 1000
    now = time.time()
    if now - start_time_q2 > max_time_limit:
        return False
    # In case both if-statements above (containing the condition that the number of steps without improvement
    # is above 100 and that the time elapsed is above 1,000 seconds) have not been satisfied, the stopping
    # conditions have not been met and the algorithm can continue. Hence, True is returned.
    return True

def create_solution_q2(previous_solution):
    """Returns a solution to question 2 by taking into consideration all relevant functions developed
    above by using those in the adequate structure."""
    # Since one stopping condition states that the running time should not exceed 1,000 seconds, the time at
    # the start of the construction algorithm is stored, to later compare the time against.
    start_time_q2 = time.time()
    # At the beginning of this algorithm, both the best and last-considered distance and solution are the those
    # given by the input solution. In the call to this function later, previous_solution is the answer to
    # question 2.1 of the assignment.
    best_state_routes = copy.deepcopy(previous_solution)
    best_state_distance = calculate_total_distance(best_state_routes)
    current_state_route = copy.deepcopy(previous_solution)
    current_state_distance = calculate_total_distance(current_state_route)
    # Since one stopping condition states that the number of steps without improvement should not exceed 100,
    # the number of steps without improvement are maintained. Initially, zero steps have been performed.
    nr_steps = 0
    # The tabu list is a Python list and since no swaps have been performed yet, it is empty.
    tabu_list = []
    # As long as the stopping condition has not been met (i.e. the function returns True), the body is executed.
    while check_stopping_condition(nr_steps, start_time_q2) == True:
        # A print-out to the screen is made in order to update the user of the progress.
        print("Nr. of steps without improvement:", nr_steps, "-- Best distance:", calculate_total_distance(best_state_routes), "-- Last distance:", current_state_distance)
        # A new_state consists of a random swap, given by the function propose_swap.
        new_state = propose_swap(current_state_route)
        # This new_state can only be implemented in case it is feasible, which is checked by check_feasibility.
        if check_feasibility(new_state[0]) == True:
            # In case the new state is feasible, there are two options: (1) the distance of the new_state is lower
            # than the current state (represented by current_state_distance) or (2) it is higher than the current_state_distance.
            if calculate_total_distance(new_state[0]) < current_state_distance:
                # In case the distance of the new_state is lower than that of the current state, the new_state is always
                # implemented. This requires setting the current state to the new state and updating the according distance.
                current_state_route = new_state[0]
                current_state_distance = calculate_total_distance(new_state[0])
                if current_state_distance < best_state_distance:
                    # In case the new state is not only better than the current state, but also better than the
                    # overall best solution, the appropriate solution and distance are updated as well.
                    best_state_routes = current_state_route
                    best_state_distance = current_state_distance
                # Since an improvement have been made, the counter for the number of steps without any
                # improvement has to be set to zero again, starting the counting all over again.
                nr_steps = 0
                # Since a swap has been performed, this swap needs to be included into the tabu list. This call
                # to update_tabu_list updates the tabu list and checks the tabu list conditions (e.g. length).
                update_tabu_list(tabu_list, new_state[1], new_state[2])
                print("IMPROVEMENT -- Last 5 insertions into tabu list:", tabu_list[:5])
            else:
                # In case the distance of new_state is higher than the current state, a swap may or may not be
                # implemented, depending on whether the swap for the new state is already in the tabu list.
                # Remember that new_state is a tuple, consisting of (set_of_routes, swapped_location1,
                # swapped_location2). The swap is implemented if the swap is not in the tabu list.
                if check_tabu_list(tabu_list, new_state[1], new_state[2]) == True:
                    # In case the swap is not in the tabu list, it should be implemented. Only the current state
                    # should be updated and not the best state, since the new state is worse than the current
                    # state. Hence, new_state cannot be the best solution found so far, since we already know
                    # of a previously encountered solution that is better.
                    current_state_route = new_state[0]
                    current_state_distance = calculate_total_distance(new_state[0])
                    # A swap is made, but since this swap does not improve the state of the solution, the counter
                    # containing the steps taken without improvement should be increased.
                    nr_steps += 1
                    # After the swap is made, the tabu list is updated accordingly.
                    update_tabu_list(tabu_list, new_state[1], new_state[2])
                    print("WORSE SWAP OCCURRED -- Last 5 insertions into tabu list:", tabu_list[:5])
                else:
                    # In case a swap is worse than the current state and is in the tabu list, it should not be
                    # implemented. Nonetheless, a step is made without an improvement. Hence, the counter
                    # containing steps without improvement should be incremented.
                    print("SWAP WAS DECLARED TABU")
                    nr_steps += 1
        else:
            # In case a state is not feasible, it is a step nonetheless. Hence, the counter containing the
            # number of steps without improvement should be incremented.
            nr_steps += 1
    # The best_state_routes is maintained throughout the algorithm and returned once the while loop has stopped.
    return best_state_routes

# The solution to question 2 is calculated by calling the create_solution_q2 function.
# Since the objective is to improve the solution of question 2.1, this solution is passed as argument.
solution_q2 = create_solution_q2(solution_q1)

Nr. of steps without improvement: 0 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 1 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 2 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 3 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 4 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 5 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 6 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 7 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 8 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 9 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 10 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps without improvement: 11 -- Best distance: 2938 -- Last distance: 2938
Nr. of steps w

# Part 3

In [ ]:
def calculate_distance_difference(current_state, new_state):
    """Calculates the difference in total distance for two given sets of routes."""
    # The distances for both the current state and the new state, i.e. the state after having swapped
    # two stores, is calculated and assigned to two variables.
    distance_current = calculate_total_distance(current_state)
    distance_new = calculate_total_distance(new_state)
    # The distance between these two is returned. In the assignment terminology, this is called Delta.
    return distance_new - distance_current

def calculate_probability(delta, temperature):
    """Calculates the probability of implementing a particular swap."""
    # The probability that a new state with the given difference from the distance of the current state
    # (i.e. Delta) is accepted is given by the calculation below. It depends on both delta and the temperature.
    return math.exp(-delta / temperature)

def check_stopping_condition_q3(temperature, start_time_q3):
    """Returns False if the stopping conditions are violated and returns True if the
    stopping conditions are not violated."""
    # There are two stopping conditions for question 2.3. First, if the temperature falls below 0.001,
    # the algorithm should stop. Then, the following if-statement is executed and this function returns
    # False, indicating that the condition has not been satisfied.
    minimum_temperature = 0.001
    if temperature < minimum_temperature:
        return False
    # The current time is given by the time method from the time module. start_time_q3 is the start time of the
    # function constructing the final answer to question 3 of the assignment. This variable is initiated by
    # time.time() as well at the beginning of the call to function create_solution_q3. Once the current time,
    # contained in variable called "now", exceeds the start time by more than 1,000 seconds, the condition is
    # violated and as a result returns False, to identify that the algorithm should be stopped.
    max_time_limit = 1000
    now = time.time()
    if now - start_time_q3 > max_time_limit:
        return False
    # In case both if-statements above (containing the condition that temperature should be below 0.001 and that
    # the time elapsed is above 1,000 seconds) have not been satisfied, the stopping conditions have not been
    # met and the algorithm can continue.
    return True

def create_solution_q3(previous_solution):
    # At the beginning of the call to this function constructing the solution to q3, the start time is recorded,
    # so that the condition that the algorithm should not run longer than 1,000 seconds can be taken care of.
    start_time_q3 = time.time()
    # Initially, the best solution and the current state consist of the output of question 2.1, which is
    # contained in the parameter previous_solution when the function is called.
    best_state_routes = copy.deepcopy(previous_solution)
    best_state_distance = calculate_total_distance(previous_solution)
    current_state_route = copy.deepcopy(previous_solution)
    current_state_distance = calculate_total_distance(current_state_route)
    # The initial temperature and cooling down-parameter are both given by the assignment and are initialised.
    # Over the course of this algorithm, the temperature will decrease at each step by the amount given by the
    # cooling down parameter.
    temperature = 1000
    cooling_down_parameter = 0.999
    while check_stopping_condition_q3(temperature, start_time_q3) == True:
        # As long as the function which checks the stopping condition returns that the algorithm can continue,
        # the algorithm prints an update about its current state and the best state seen so far.
        print("Temperature:", round(temperature, 4), "-- Best distance so far:", best_state_distance, "-- Last distance:", current_state_distance)
        # A new state is considered by calling the function propose_swap, which gives us a new set of routes,
        # in which two stores have been swapped. This new state may or may not be feasible. Note that
        # new_state is a tuple containing 3 elements: (set_of_routes, swapped_store_1, swapped_store_2).
        new_state = propose_swap(current_state_route)
        # After a new state has been developed, its feasibility is checked.
        if check_feasibility(new_state[0]) == True:
            # Once the new state has been checked for feasibility and has been deemed feasible, it is further
            # considered. Delta measures the difference between the distance of the current state and the
            # distance of the new state in which two stores have been swapped.
            delta = calculate_distance_difference(current_state_route, new_state[0])
            if delta < 0:
                # If delta is lower than zero, the distance for the new state is lower than the distance of
                # the current state. Hence, a swap takes place and the current state is updated.
                print("Delta lower than 0, there is an improvement.")
                current_state_route = new_state[0]
                current_state_distance = calculate_total_distance(new_state[0])
                # If the new state is not only better than the current state, but also better than the best state
                # overall, the variables storing the best state need to be updated in the if-statement below.
                if current_state_distance < best_state_distance:
                    best_state_routes = new_state[0]
                    best_state_distance = calculate_total_distance(best_state_routes)
                # Once the current and possibly the best state have been updated, the temperature is decremented
                # by the amount given by the cooling_down_parameter.
                temperature -= cooling_down_parameter
            else:
                # In case delta is not below zero, the new state containing a swapped set of routes is only
                # implemented with a given probability. That probability is calculated by calling the function
                # below. The subsequent if-statement draws a random number between 0 and 1. If this number is
                # equal to or lower than the probability of acceptance, the current state will be updated.
                probability_implementation = calculate_probability(delta, temperature)
                if random.uniform(0, 1) <= probability_implementation:
                    print("A swap occured. The probability for acceptance was:", probability_implementation)
                    # In order to implement a swap, the variables containing the current state are updated.
                    current_state_route = new_state[0]
                    current_state_distance = calculate_total_distance(new_state[0])
                    # In contrast to the situation where the delta is lower than zero, there is no need to
                    # check whether the new_state is better than the best state. Since the new_state is
                    # worse than the current_state, we know that it cannot be the best state, since we already
                    # know of a state that is better, which is the current state. After the swap has been
                    # implemented, the temperature is decremented by the cooling_down_parameter.
                    temperature -= cooling_down_parameter
                else:
                    # In case the swap is not accepted, the temperature is still decremented.
#                     temperature -= cooling_down_parameter
                    continue
        else:
            # In case a swap has been deemed unfeasible, the temperature is still decremented.
#             temperature -= cooling_down_parameter
            continue
    # The best state is maintained throughout the code and is returned at the end of the algorithm.
    return best_state_routes

# The solution to question 3 is calculated by calling the create_solution_q3 function.
# Since the objective is to improve the solution of question 2.1, this solution is passed as argument.
solution_q3 = create_solution_q3(solution_q1)

Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
Temperature: 1000 -- Best distance so far: 2938 -- Last distance: 2938
A swap occured. The probability for acceptance was: 0.9880717128619305
Temperature: 999.001 -- Best distance so far: 2938 -- Last distance: 2950
Temperature: 999.001 -- Best distance so far: 2938 -- Last distance: 2950


## Writing solutions to Excel file

In [ ]:
# The final solutions are stored in the variables solution_q1, solution_q2 and solution_q3.
# The following functions then creates Excel files and writes these solutions to that file.
import xlsxwriter

def add_headquarters_to_route(set_of_routes):
    """Adds a headquarter at the beginning and end of the route. It does not check the feasibility of
    these additions, since this is already done during the creation of the routes itself."""
    # In order to prevent changes to the original set_of_routes, a deepcopy is made.
    new_routes = copy.deepcopy(set_of_routes)
    for route in new_routes:
        # For every route within the set of routes, a 0 (indicating the City Nr. for the headquarter)
        # is added to the beginning and the end of the route. The routes should not be checked for
        # feasibility again, since this is already done during the construction of the routes.
        route.insert(0, 0)
        route.insert(len(route), 0)
    return new_routes

def write_to_excel(filename, question_result):
    """Given a filename and a set of routes as a nested list, this function returns
    an excel file that complies with the requirements set forth by the assignment."""
    # In order to write the data to the Excel file, the lists of routes are flattened to a single list.
    final_result = add_headquarters_to_route(question_result)
    # The worksheet is instantiated and column names are assigned to the columns in the Excel file.
    excel_workbook = xlsxwriter.Workbook(str(filename) + ".xls")
    excel_sheet = excel_workbook.add_worksheet()
    excel_sheet.write("A1", "Route Nr.")
    excel_sheet.write("B1", "City Nr.")
    excel_sheet.write("C1", "City Name")
    excel_sheet.write("D1", "Total Distance in Route (km)")
    excel_sheet.write("E1", "Total distance (km)")
    # Total distance will contain the distances between all the individual stores in the routes. Later,
    # this list is flattened and made cumulative, which then represents the total distances over all routes.
    # In the Excel file, this will ultimately become the column "Total Distance (km)".
    total_distance = []
    # total_distance_in_route will contain the cumulative distance within every route of the set of
    # routes. In the Excel file, this will ultimately become the column "Total Distance in Route (km)".
    total_distance_in_route = []
    
    for route in final_result:
        # By iterating over the set of routes, each route is transformed into destination pairs, which 
        # means that a route [A,B,C,D] is transformed into a list [(A,B),(B,C),(C,D)].
        destination_pairs = list(zip(route, route[1:]))
        # Since the distance is initially zero, the distances_route list initially contains 0 km.
        distances_route = [0]
        for pair in destination_pairs:
            # When iterating over the destination pairs, the distances between the elements of these pairs
            # are added to distance_route, which will contain the distances between the subsequent locations
            # within the particular route.
            distances_route.append(round(distance_between_supermarkets(pair[0], pair[1])))
        # When all decision pairs are treated, the list is added to the overall list total_distance. As a
        # result, total_distance will consist of a list of lists, containing the distances between the
        # locations of the routes that it reflects.
        total_distance.append(distances_route)
        # The distances are appended to total_distance_in_route in cumulative form, which makes that this
        # list total_distance_in_route becomes a list of lists containing the cumulative distances for
        # all the routes in the final solution.
        total_distance_in_route.append(np.cumsum(distances_route))
    # In order to insert the data effectively into the Excel file, the lists are flattened, which means
    # that the nested lists will become a single list with all individual lists concatenated.
    flattened_total_distance = [store for day in total_distance for store in day]
    flattened_total_distance_in_route = [store for day in total_distance_in_route for store in day]
    flattened_total_distance_cumulative = np.cumsum(flattened_total_distance)
    
    # The data is written to the Excel file by iterating over the rows of the file. row_counter keeps track
    # of the row that data is currently written to. route_counter assigns a route number to the routes
    # and iterates on the nested list level.
    row_counter = 1
    route_counter = 1
    for route in final_result:
        for location in route:
            excel_sheet.write(row_counter, 0, route_counter)
            excel_sheet.write(row_counter, 1, location)
            excel_sheet.write(row_counter, 2, dataset.loc[dataset["City Nr."] == location]["Name"].to_string(index=False).strip())
            row_counter += 1
        route_counter += 1
    row_counter = 1
    for index in range(len(flattened_total_distance_in_route)):
        excel_sheet.write(row_counter, 3, flattened_total_distance_in_route[index])
        row_counter += 1
    row_counter = 1
    for index in range(len(flattened_total_distance_cumulative)):
        excel_sheet.write(row_counter, 4, flattened_total_distance_cumulative[index])
        row_counter += 1
    excel_workbook.close()

# Finally, the function is called on the three separate solutions to create three separate Excel files.
# These files are saved in the same directory as this file.
write_to_excel("Ex2.1-1273432", solution_q1)
write_to_excel("Ex2.2-1273432", solution_q2)
write_to_excel("Ex2.3-1273432", solution_q3)